# Week 3 - Targets for DSB

Comparing Transactions


In [1]:
#Import libraries
import pandas as pd

In [2]:
#Import file
transactions = pd.read_csv(r'/Users/KellyLam/Desktop/Data_Analysis/Data_Cleaning/PD 2023 Wk 1 Input.csv')
target = pd.read_csv(r'/Users/KellyLam/Desktop/Data_Analysis/Data_Cleaning/Targets.csv')

In [3]:
#Filter transactions for DSB
transactions_dsb = transactions[transactions['Transaction Code'].str.contains('DSB')]
transactions_dsb = pd.DataFrame(transactions_dsb)
transactions_dsb.head()

,Transaction Code,Value,Customer Code,Online or In-Person,Transaction Date
2,DSB-807-592-406,5520,100005,1,14/07/2023 00:00:00
4,DSB-474-374-857,5375,100000,2,26/08/2023 00:00:00
5,DSB-448-546-348,4525,100009,1,27/05/2023 00:00:00
11,DSB-422-218-322,118,100010,1,12/05/2023 00:00:00
12,DSB-669-227-170,830,100001,1,15/04/2023 00:00:00


In [4]:
#Rename values to 1 and 2 for Online or In-Person Column
transactions_dsb["Online or In-Person"] = transactions_dsb["Online or In-Person"].replace({1 : "Online", 2: "In-Person"})
transactions_dsb.head()

,Transaction Code,Value,Customer Code,Online or In-Person,Transaction Date
2,DSB-807-592-406,5520,100005,Online,14/07/2023 00:00:00
4,DSB-474-374-857,5375,100000,In-Person,26/08/2023 00:00:00
5,DSB-448-546-348,4525,100009,Online,27/05/2023 00:00:00
11,DSB-422-218-322,118,100010,Online,12/05/2023 00:00:00
12,DSB-669-227-170,830,100001,Online,15/04/2023 00:00:00


In [5]:
#Change date to quarter
transactions_dsb.info() #checking for datetime type for Transaction Date column
#Converting Transaction Date column to datetime type
transactions_dsb["Quarter"] = pd.to_datetime(transactions_dsb["Transaction Date"], format="%d/%m/%Y %H:%M:%S").dt.quarter
transactions_dsb = transactions_dsb.rename(columns={"Quarter": "Quarter"})
transactions_dsb.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102 entries, 2 to 359
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Transaction Code     102 non-null    object
 1   Value                102 non-null    int64 
 2   Customer Code        102 non-null    int64 
 3   Online or In-Person  102 non-null    object
 4   Transaction Date     102 non-null    object
dtypes: int64(2), object(3)
memory usage: 4.8+ KB


,Transaction Code,Value,Customer Code,Online or In-Person,Transaction Date,Quarter
2,DSB-807-592-406,5520,100005,Online,14/07/2023 00:00:00,3
4,DSB-474-374-857,5375,100000,In-Person,26/08/2023 00:00:00,3
5,DSB-448-546-348,4525,100009,Online,27/05/2023 00:00:00,2
11,DSB-422-218-322,118,100010,Online,12/05/2023 00:00:00,2
12,DSB-669-227-170,830,100001,Online,15/04/2023 00:00:00,2


In [6]:
#Sum the transaction values for each quarter and for each Type of Transaction (Online or In-Person)
transactions_sum = transactions_dsb[["Online or In-Person", "Quarter", "Value"]].groupby(["Online or In-Person", "Quarter"]).sum().reset_index()
transactions_sum.head()

,Online or In-Person,Quarter,Value
0,In-Person,1,77576
1,In-Person,2,70634
2,In-Person,3,74189
3,In-Person,4,43223
4,Online,1,74562


In [7]:
#Pivot the quarterly targets to have a row for each Type of Transactoin and each Quarter
target.head()


,Online or In-Person,Q1,Q2,Q3,Q4
0,Online,72500,70000,60000,60000
1,In-Person,75000,70000,70000,60000


In [8]:
target_pivot = pd.melt(target, id_vars='Online or In-Person', var_name = 'Quarter', value_name = 'Target')
target_pivot = pd.DataFrame(target_pivot)
target_pivot.head()

,Online or In-Person,Quarter,Target
0,Online,Q1,72500
1,In-Person,Q1,75000
2,Online,Q2,70000
3,In-Person,Q2,70000
4,Online,Q3,60000


In [9]:
#Removing Q from Quarter Column
target_pivot['Quarter'] = target_pivot['Quarter'].str.replace('Q','')

#Converting Quarter Column to Numeric values
target_pivot['Quarter'] = pd.to_numeric(target_pivot['Quarter'])
target_pivot.head()

,Online or In-Person,Quarter,Target
0,Online,1,72500
1,In-Person,1,75000
2,Online,2,70000
3,In-Person,2,70000
4,Online,3,60000


In [10]:
#Joining tables
output_df = pd.merge(transactions_sum, target_pivot, on=['Quarter', 'Online or In-Person'])

In [11]:
#Renaming Targets to Quarterly Targets
output_df = output_df.rename(columns={'Target': 'Quarterly Targets'})
#Reordering columns
output_df = output_df[['Online or In-Person', 'Quarter', 'Value', 'Quarterly Targets']]
#Calculate Variance to Target
output_df['Variance to Target'] = output_df['Value'] - output_df['Quarterly Targets']
output_df

,Online or In-Person,Quarter,Value,Quarterly Targets,Variance to Target
0,In-Person,1,77576,75000,2576
1,In-Person,2,70634,70000,634
2,In-Person,3,74189,70000,4189
3,In-Person,4,43223,60000,-16777
4,Online,1,74562,72500,2062
5,Online,2,69325,70000,-675
6,Online,3,59072,60000,-928
7,Online,4,61908,60000,1908


In [12]:
#Exporting each dataframe to CSV
folder_path = '/Users/KellyLam/Desktop/Data_Analysis/Data_Cleaning/'
file_name = 'dsb_targets.csv'
file_path = folder_path + file_name

output_df.to_csv(file_path, index=False)
